<a href="https://colab.research.google.com/github/rajyash1904/BTP_PKB-2/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# New project

In [3]:
import os
import time
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
import pandas as pd
#tf.enable_eager_execution()

In [4]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


Set GPU/CPU

In [6]:
# OS.ENVIRON['CUDA_VISIBLE_DEVICES']="" #ONLY CPU

#config = tf.configProto()
config = tf.compat.v1.ConfigProto
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
config.log_devices_placement = True
#config.device_count={'gpu':0}
sess = tf.Session(config=config)

AttributeError: '_FieldProperty' object has no attribute 'per_process_gpu_memory_fraction'

## COMPRESSION


Pre-Process

In [7]:
from process import preprocess

In [8]:
input_file = 'testdata/8iVFB/longdress_vox10_1300.ply'
scale = 1
cube_size = 64
min_num = 64
res = 1024

In [9]:
cubes, cube_positions, poins_numbers = preprocess(input_file, scale, cube_size, min_num)

==== Preprocess ====
Scaling: 0.0s


NameError: name 'load_points' is not defined

In [ ]:
fig = plt.figure(figsize=(10,2))
print('point_numbers (min/max/mean):', points_numbers.min(), point_numbers.max(), int(points_numbers.mean()))
plt.title('points numbers')
n, bins, patches = plt.hist(points_numbers, bins=256, range=(1, pow(2,16)))
plt.show()
